<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d885e0cfd95c6a7f441c73437ab724a379a5853710c48ec06cbadbae63645ba4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-06 09:27:59
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: -39.08 K (-0.27%)
Current PnL: -25.33 L (-16.15%)
CY Booked + Current PnL: -11.04 L (-7.04%)
-------------------
Total profit:  1.94 L
Total loss:  -27.27 L
-------------------
Total Booked + Current PnL: 15.94 L (12.28%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.94%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 90.30 L (62.82%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,0.02,15.55,3.58,19.69,7932.0,29823.0,221565.0,4.31,70.0,X-MC,3.56,78.0,3.76,1.54,23.41,XY25,NTT,AC
51,NESTLEIND,1377.00,-0.40,15.43,5.17,21.40,15840.0,40950.0,306376.0,7.09,67.0,X-LC,5.00,12.0,2.59,2.13,22.73,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.85,-19.33,24.07,0.09,19569.0,-19478.0,81299.0,88.69,44.0,M-SC,3.26,253.0,-1.00,0.57,6.81,OX40N,NTT,DURABLES
17,COALINDIA,484.83,0.65,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,84.0,L-LC,5.70,179.0,0.43,1.11,28.31,XY25,ATH,MINING
49,MASFIN,398.61,1.75,-0.14,22.22,22.05,21741.0,-135.0,97845.0,-13.73,57.0,H-SC,6.42,164.0,-0.01,0.68,42.22,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,7.47,-27.02,91.31,39.61,68300.0,-27700.0,74800.0,-50.65,65.0,H-SC,32.32,135.0,-0.41,0.52,21.07,AR,ATH,MISC
49,MASFIN,398.61,1.75,-0.14,22.22,22.05,21741.0,-135.0,97845.0,-13.73,57.0,H-SC,6.42,164.0,-0.01,0.68,42.22,XR,ATH,FINANCE
36,ICICIPRULI,790.00,1.74,15.13,13.55,30.73,30733.0,29809.0,226814.0,-11.75,79.0,X-MC,6.00,75.0,0.97,1.58,29.83,X40,ATH,INSURANCE
5,ASIANPAINT,3460.25,0.74,-0.86,21.99,20.95,54888.0,-2165.0,249603.0,-1.90,58.0,X-LC,12.07,31.0,-0.04,1.74,34.47,X40,ATH,PAINTS
17,COALINDIA,484.83,0.65,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,84.0,L-LC,5.70,179.0,0.43,1.11,28.31,XY25,ATH,MINING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1952.0,-3.52,-2.27,28.21,25.30,55405.0,-4561.0,196402.0,-5.38,42.0,X-LC,9.02,17.0,-0.08,1.37,31.56,XY25,BTT,REFINERIES
75,TMPV,600.0,-2.62,-25.45,64.95,22.98,109386.0,-57482.0,168416.0,-23.08,51.0,X-LC,5.03,3.0,-0.53,1.17,5.93,XY24,NTT,AUTO
0,5PAISA,593.0,-1.70,-29.71,60.62,12.90,93104.0,-64909.0,153587.0,125.96,66.0,H-SC,11.07,161.0,-0.70,1.07,27.71,OX40N,NTT,FINANCE
83,WHIRLPOOL,2270.0,-1.59,-27.79,151.94,81.92,125946.0,-31906.0,82892.0,-57.77,34.0,M-SC,24.66,236.0,-0.25,0.58,1.38,XR,NTT,DURABLES
79,VAIBHAVGBL,521.0,-1.43,-28.76,125.67,60.77,163636.0,-52564.0,130211.0,50.41,43.0,H-SC,4.99,159.0,-0.32,0.91,17.26,XR,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,1.75,-0.14,22.22,22.05,21741.0,-135.0,97845.0,-13.73,57.0,H-SC,6.42,164.0,-0.01,0.68,42.22,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.23,-1.19,110.41,107.91,161075.0,-1756.0,145888.0,-19.74,56.0,M-SC,10.24,216.0,-0.01,1.02,8.78,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.96,-16.65,19.99,0.02,29086.0,-29057.0,145502.0,122.10,48.0,M-SC,7.21,234.0,-1.00,1.01,26.56,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.85,-19.33,24.07,0.09,19569.0,-19478.0,81299.0,88.69,44.0,M-SC,3.26,253.0,-1.00,0.57,6.81,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.09,-21.00,43.64,13.47,92966.0,-56637.0,213030.0,-67.53,54.0,H-SC,5.37,158.0,-0.61,1.48,8.32,XY24,NTT,PAINTS
66,SIS,528.00,-0.59,-22.76,57.00,21.26,48881.0,-25276.0,85756.0,2024.87,54.0,H-SC,4.31,166.0,-0.52,0.60,15.87,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.39,3.55,16.60,20.74,34922.0,7205.0,210375.0,35.83,74.0,M-MC,6.86,189.0,0.21,1.46,14.54,XY25,NTT,FINANCE
17,COALINDIA,484.83,0.65,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,84.0,L-LC,5.70,179.0,0.43,1.11,28.31,XY25,ATH,MINING
84,WIPRO,420.00,0.63,8.67,58.52,72.27,107759.0,14692.0,184140.0,-6.53,56.0,M-LC,5.81,99.0,0.14,1.28,15.38,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.63,8.67,58.52,72.27,107759.0,14692.0,184140.0,-6.53,56.0,M-LC,5.81,99.0,0.14,1.28,15.38,XR,NTT,IT
49,MASFIN,398.61,1.75,-0.14,22.22,22.05,21741.0,-135.0,97845.0,-13.73,57.0,H-SC,6.42,164.0,-0.01,0.68,42.22,XR,ATH,FINANCE
25,FINCABLES,1641.55,-0.23,-1.19,110.41,107.91,161075.0,-1756.0,145888.0,-19.74,56.0,M-SC,10.24,216.0,-0.01,1.02,8.78,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.10,-6.54,121.18,106.72,139690.0,-8061.0,115275.0,-53.68,38.0,H-SC,6.86,139.0,-0.06,0.80,15.31,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.29,-3.06,39.92,35.63,81099.0,-6420.0,203154.0,-16.74,47.0,H-MC,3.02,119.0,-0.08,1.41,13.10,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.30,-14.55,29.64,10.78,60867.0,-34963.0,205355.0,-48.92,14.0,X-LC,4.18,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
23,DMART,5391.45,-0.81,-13.06,49.08,29.60,106502.0,-32604.0,216996.0,-25.88,25.0,X-LC,9.72,19.0,-0.31,1.51,6.25,X40N,ATH,FMCG
22,DIXON,18931.72,-0.55,-16.90,57.96,31.26,111144.0,-39008.0,191760.0,-61.22,31.0,X-MC,17.96,56.0,-0.35,1.33,1.85,X40N,ATH,IT
7,AWL,485.00,-0.17,-24.35,107.80,57.19,265947.0,-79423.0,246704.0,-64.27,32.0,X-MC,8.07,58.0,-0.30,1.72,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,-0.42,-10.40,45.78,30.62,97237.0,-24654.0,212400.0,-36.35,33.0,X-MC,15.84,55.0,-0.25,1.48,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,-0.08,0.96,19.62,20.77,46381.0,2253.0,236397.0,-7.76,38.0,X-MC,0.94,62.0,0.05,1.64,7.95,X40N,NTT,BREWERIES
65,SIEMENS,4671.5,-0.05,-17.11,51.43,25.51,79331.0,-31845.0,154250.0,-5.41,44.0,H-LC,1.38,49.0,-0.40,1.07,13.67,AR,ATH,ELECTRICAL
20,DABUR,735.0,-1.20,0.68,42.73,43.70,106719.0,1683.0,249751.0,-7.59,60.0,X-MC,1.93,72.0,0.02,1.74,15.83,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-0.58,-6.37,29.39,21.14,62556.0,-14491.0,212848.0,-13.02,40.0,X-MC,2.28,74.0,-0.23,1.48,18.20,XY24,NTT,PAINTS
45,JIOFIN,387.0,0.08,-3.29,28.72,24.48,74258.0,-8806.0,258559.0,-4.90,53.0,X-LC,2.31,36.0,-0.12,1.80,49.87,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.17,-24.35,107.80,57.19,265947.0,-79423.0,246704.0,-64.27,32.0,X-MC,8.07,58.0,-0.30,1.72,0.00,XY24,NTT,FMCG
43,ITC,452.00,-0.30,-14.55,29.64,10.78,60867.0,-34963.0,205355.0,-48.92,14.0,X-LC,4.18,5.0,-0.57,1.43,0.00,X40,NTT,FMCG
52,PAGEIND,51605.07,-0.42,-10.40,45.78,30.62,97237.0,-24654.0,212400.0,-36.35,33.0,X-MC,15.84,55.0,-0.25,1.48,0.00,X40,ATH,APPARELS
58,RELAXO,1176.00,-0.92,-47.87,192.17,52.31,145415.0,-69490.0,75670.0,-44.54,44.0,X-SC,7.40,91.0,-0.48,0.53,1.29,X40N,NTT,FOOTWEAR
19,COLPAL,3726.84,-0.49,-20.98,79.08,41.51,164573.0,-55255.0,208110.0,-5.86,40.0,X-MC,8.72,61.0,-0.34,1.45,1.35,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.61,-28.67,100.26,42.84,46490.0,-18637.0,46369.0,-53.17,49.0,X-SC,29.31,83.0,-0.40,0.32,4.91,XY24,NTT,MISC
58,RELAXO,1176.00,-0.92,-47.87,192.17,52.31,145415.0,-69490.0,75670.0,-44.54,44.0,X-SC,7.40,91.0,-0.48,0.53,1.29,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.12,-38.33,119.25,35.20,94622.0,-49322.0,79348.0,3.97,44.0,X-SC,17.20,92.0,-0.52,0.55,1.91,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-0.31,3.00,20.46,24.08,26754.0,3812.0,130762.0,-5.33,61.0,X-SC,6.93,89.0,0.14,0.91,23.21,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.06,-15.64,69.89,43.32,96029.0,-25472.0,137400.0,-27.57,55.0,X-SC,7.75,90.0,-0.27,0.96,5.50,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.23,-11.37,36.19,20.71,121321.0,-43003.0,335234.0,-23.62,50.0,X-LC,2.63,1.0,-0.35,2.33,12.01,X40,ATH,IT
41,INFY,1972.00,0.39,9.79,22.28,34.24,76533.0,30617.0,343505.0,-14.57,50.0,X-LC,7.46,2.0,0.40,2.39,18.88,X40,NTT,IT
75,TMPV,600.00,-2.62,-25.45,64.95,22.98,109386.0,-57482.0,168416.0,-23.08,51.0,X-LC,5.03,3.0,-0.53,1.17,5.93,XY24,NTT,AUTO
81,VBL,671.64,-0.31,-1.59,37.86,35.67,117682.0,-5008.0,310834.0,-13.27,57.0,X-LC,2.41,4.0,-0.04,2.16,12.05,X40N,ATH,FMCG
43,ITC,452.00,-0.30,-14.55,29.64,10.78,60867.0,-34963.0,205355.0,-48.92,14.0,X-LC,4.18,5.0,-0.57,1.43,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.28,-33.74,100.57,32.89,51447.0,-26052.0,51155.0,-731.43,68.0,L-MC,7.09,259.0,-0.51,0.36,40.88,XR,NTT,BANKS
49,MASFIN,398.61,1.75,-0.14,22.22,22.05,21741.0,-135.0,97845.0,-13.73,57.0,H-SC,6.42,164.0,-0.01,0.68,42.22,XR,ATH,FINANCE
13,BSOFT,831.70,-0.03,-22.58,94.75,50.79,103181.0,-31755.0,108898.0,-0.66,51.0,H-SC,10.76,151.0,-0.31,0.76,26.79,XR,ATH,IT
39,INDIGOPNTS,1408.00,-0.96,-16.65,19.99,0.02,29086.0,-29057.0,145502.0,122.10,48.0,M-SC,7.21,234.0,-1.00,1.01,26.56,OX40N,NTT,PAINTS
69,SURYODAY,216.00,-0.35,-19.00,51.90,23.03,75278.0,-34027.0,145044.0,57.89,54.0,H-SC,5.33,167.0,-0.45,1.01,43.91,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.70,-29.71,60.62,12.90,93104.0,-64909.0,153587.0,125.96,66.0,H-SC,11.07,161.0,-0.70,1.07,27.71,OX40N,NTT,FINANCE
74,TITAGARH,1548.00,-0.51,-17.26,76.14,45.74,152568.0,-41799.0,200378.0,10.32,59.0,H-SC,2.87,174.0,-0.27,1.39,30.33,XY24,NTT,ENGINEERING
17,COALINDIA,484.83,0.65,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,84.0,L-LC,5.70,179.0,0.43,1.11,28.31,XY25,ATH,MINING
36,ICICIPRULI,790.00,1.74,15.13,13.55,30.73,30733.0,29809.0,226814.0,-11.75,79.0,X-MC,6.00,75.0,0.97,1.58,29.83,X40,ATH,INSURANCE
56,RAJOOENG,143.10,7.47,-27.02,91.31,39.61,68300.0,-27700.0,74800.0,-50.65,65.0,H-SC,32.32,135.0,-0.41,0.52,21.07,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.02
1,25,44.07
2,50,76.39


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.91
MC    30.89
LC    26.24
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.67
X40      23.87
X40N     12.74
XY25     10.71
XR        8.95
AR        8.62
OX40N     7.52
SR        0.96
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.41
H-SC    22.83
X-LC    22.78
M-SC    11.36
X-SC     7.93
H-MC     4.66
M-MC     1.46
M-LC     1.28
L-LC     1.11
H-LC     1.07
L-SC     0.79
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.93,-8.29,43.82
IT,12.66,-17.40,76.62
FINANCE,11.16,-11.17,57.83
MISC,7.08,-27.67,79.90
ELECTRICAL,5.89,-10.94,51.58
PAINTS,5.71,-12.47,29.17
INSURANCE,4.78,1.06,32.80
PHARMA,3.94,-3.46,36.04
AUTO,2.83,-28.71,73.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3137694.0,21
AR,1331581.0,10
XR,1208055.0,12
X40,1090554.0,15
X40N,912317.0,9
OX40N,713466.0,10
XY25,353588.0,7
SR,282690.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3515135.0,24
X-MC,1477459.0,16
M-SC,1428077.0,15
X-LC,948943.0,13
X-SC,780939.0,8
H-MC,408575.0,3
L-SC,177157.0,2
M-LC,107759.0,1
H-LC,79331.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1223734.0      6
           AR         910922.0      5
M-SC       XY24       813133.0      6
H-SC       XR         784271.0      7
X-MC       X40        488285.0      7
           XY24       435222.0      3
X-LC       X40        415155.0      6
X-MC       X40N       381447.0      4
H-SC       OX40N      313518.0      4
M-SC       OX40N      308789.0      5
X-SC       X40N       306686.0      3
           XY24       287139.0      3
H-SC       SR         282690.0      2
X-LC       X40N       224184.0      2
H-MC       AR         213753.0      2
           XY24       194822.0      1
X-SC       X40        187114.0      2
X-LC       XY24       183644.0      2
M-SC       XR         178580.0      2
X-MC       XY25       172505.0      2
M-SC       AR         127575.0      2
X-LC       XY25       125960.0      3
M-LC       XR         107759.0      1
L-SC       OX40N       91159.0      1
           XR          85998.0      1
H-LC       AR          79331.0      1
L-MC       XR          51447.0      1
M-MC       XY25        34922.0      1
L-LC       XY25        20201.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
